In [1]:
import copy

In [2]:
# Edge data structure
class Edge:

    def __init__(self, node, weight):          
        self.node   = node
        self.weight = weight

    def to(self):                                  
        return self.node
    
    def cost(self):                                  
        return self.weight

In [3]:
# Node data structure
class Node: 

    def __init__(self, state, cost=0,move = "", total_cost=0, goal = False):        
        self.out_edges = []
        self.state = state
        #self.label = state[0]
        self.is_goal = goal
        self.cost = cost
        self.total_cost = total_cost
        self.move   = move



    def add_edge(self, node, weight):          
        self.out_edges.append(Edge(node, weight))

In [4]:
def insert_queu(priority_queue2, data):    
    if not priority_queue2:
        priority_queue2.append(data)

    elif len(priority_queue2) == 1: 
        if priority_queue2[0][0].cost < data[0].cost : 
            priority_queue2.append(data)
        else: priority_queue2.insert(0, data)    

    else:
        index = -1
        for j, d in enumerate(priority_queue2):                      # Searching for the position
            if priority_queue2[j][0].state == data[0].state:      # don't add node added the previous to queue 
                if priority_queue2[j][0].cost > data[0].cost:     # select the node the chepest the cost 
                    priority_queue2.pop(j)
                else:
                    return;
       
        for i in range(len(priority_queue2)):                     # Searching for the position
            if priority_queue2[i][0].cost > data[0].cost:
                index = i
                break
        if index == -1:
            priority_queue2.append(data)
        else:
            priority_queue2.insert(i, data)          

In [5]:
def add_child(stacks2, disk, cost_out, id_source, node, id_destination, h, goal_stack):
    new_state = copy.deepcopy(stacks2)
    new_state[id_destination][1].append(disk)
    cost_in = new_state[id_destination][2]
    wighte = cost_out + cost_in
    move_disks = "Move disk "+str(disk)+" from source stack "+str(id_source)+" to destination stack "+\
                str(id_destination)+" the cost "+str(node.cost +wighte) 
    child = Node(new_state, node.cost + wighte, move_disks)
    if h == True:
        child.total_cost = child.cost + heuristic(child,goal_stack)
    node.add_edge(child ,wighte)
    

In [6]:
def expansion(node, h = False, goal_stack=0 ):
        stacks = copy.deepcopy(node.state)
        for stack in stacks:       
            if not stack[1]:
                continue
            id_stack, disks_of_stack, cost_out = stack
            if len(disks_of_stack) > 0:
                stacks2 = copy.deepcopy(node.state)
                disk = stacks2[id_stack][1].pop()       

            for stack2 in stacks2:
                if stack2[0] == id_stack:
                    continue
                elif  not stack2[1]:               
                    add_child(stacks2, disk, cost_out, id_stack, node, stack2[0], h,goal_stack )
                elif stack2[1][-1] < disk: 
                    add_child(stacks2, disk, cost_out, id_stack, node, stack2[0], h, goal_stack)

In [7]:
def ucs(start, goal):
    visited = []                             # set of visited nodes
    priority_queue = []                      # we store vertices in the (priority) queue as tuples 
                                             # (cost, n, path), with
                                             # cost: the cumulative cost,
                                             # n: the current node,


    data =  (start, [start])              # path: the path that led to the expansion of the current node
    insert_queu(priority_queue, data)             # add the starting node, this has zero *cumulative* cost 
                                                  # and it's path contains only itself. Also sorting in queu
    number_the_node_expanded = 0
    while not (not priority_queue):               # while the queue is nonempty and pop min cost
        current_node, path = priority_queue.pop(0) 
        visited.append(current_node.state)         # mark node visited on expansion,
                                                   # only now we know we are on the cheapest path to the current node.


        expansion(current_node)

#             print(current_node.state,current_node.cost)    
        if current_node.state == goal.state:             # if the current node is a goal
            print("***************************")
            print("The goal state has been successfully reached")
            print("the number expanded node:",len(visited),"the cost:",current_node.cost )
            print("the state start:", start.state)
            print("the curent state is a goal:", current_node.state)
            f = open("Output_Movements .txt", "a")
                
            for node in path:
                print(node.state, "the cost ",node.cost)

            for node in path:    
                print(node.move)                   
                f.write(node.move)
            f.close()                   # return its path
        else: 
            for edge in current_node.out_edges:
                child = edge.to()
                if child.state not in visited:         #  Don't add any node if it was previously visited 
                    data = (child, path + [child]) # (cost the node, node, path in root to node)
                    insert_queu(priority_queue, data)
    print("Reach did the optimal goal state")
    # younes Albetawi

In [8]:
def insert_queu_h(priority_queue, data):    
    if not priority_queue:
        priority_queue.append(data)

    elif len(priority_queue) == 1: 
        if priority_queue[0][0].total_cost < data[0].total_cost : 
            priority_queue.append(data)
        else: priority_queue.insert(0, data)    

    else:
        index = -1
        for j, d in enumerate(priority_queue):                      # Searching for the position
            if priority_queue[j][0].state == data[0].state:      # don't add node added the previous to queue 
                if priority_queue[j][0].total_cost > data[0].total_cost:     # select the node the chepest the cost 
                    priority_queue.pop(j)
                else:
                    return;
       
        for i in range(len(priority_queue)):                     # Searching for the position
            if priority_queue[i][0].total_cost > data[0].total_cost:
                index = i
                break
        if index == -1:
            priority_queue.append(data)
        else:
            priority_queue.insert(i, data)          

In [9]:
def heuristic(node, goal_stack):
    initial_state = node.state
    heuristic_value = 0
    # number the move * cost 
    #
    for index, stack in enumerate(initial_state):
                
        if not stack[1]:
            continue
        if index == goal_stack:
            for i,disk in enumerate(stack[1]):
                if i+1 != disk:
                    heuristic_value +=2
                if i+1 == disk:
                    heuristic_value -=1
        else:
            heuristic_value += 2 ** (len(stack[1])-1)
    
    return heuristic_value*2; 

In [10]:
def A_star_tree(start, goal, goal_stack):
    visited_h = []                             # set of visited nodes
    priority_queue_h = []                      # we store vertices in the (priority) queue as tuples 
                                             # (cost, n, path), with
                                             # cost: the cumulative cost,
                                             # n: the current node,


    data =  (start, [start])              # path: the path that led to the expansion of the current node
    insert_queu_h(priority_queue_h, data)             # add the starting node, this has zero *cumulative* cost 
                                                  # and it's path contains only itself. Also sorting in queu
    number_the_node_expanded = 0
    while not (not priority_queue_h):               # while the queue is nonempty and pop min cost
        current_node, path = priority_queue_h.pop(0) 
        visited_h.append(current_node.state)         # mark node visited on expansion,
                                                   # only now we know we are on the cheapest path to the current node.

        expansion(current_node, True, goal_stack )

        if current_node.state == goal.state:             # if the current node is a goal
            print("***************************")
            print("The goal state has been successfully reached")
            print("the number expanded node:",len(visited_h),"the cost:",current_node.total_cost )
            print("the state start:", start.state)
            print("the curent state is a goal:", current_node.state)
            f = open("Output_Movements .txt", "a")
                
            for node in path:
                print(node.state)

            for node in path:    
                print(node.move)                   
                f.write(node.move)
            f.close()
            return path                      # return its path
        else: 
            for edge in current_node.out_edges:
                child = edge.to()
                if child.state not in visited_h:         #  Don't add any node if it was previously visited 
                    data = (child, path + [child]) # (cost the node, node, path in root to node)
                    insert_queu_h(priority_queue_h, data)
    print("Reach did the optimal goal state")
    # younes Albetawi

In [11]:
def input_stacks():
    number_stake   = int(input("Enter the number of stacks: ")) # parse string into an integer
    number_of_disk = int(input("Enter the number of disks: "))  # parse string into an integer

    cost_of_stake = [int(input("Enter cost: ")) for i in range(number_stake)] 
    staks = [[i,[]] for i in range(0, number_stake)]
    for i in range(len(cost_of_stake)):
        staks[i].append(cost_of_stake[i])
    
    print(staks)
    print("[id of stak, [disk], cost input or output] == ",staks[0])
    ini_staks = copy.deepcopy(staks)
    initioal_stack= int(input("Which stack begins the initial state: "))
    ini_staks[initioal_stack][1] = [i for i in range(1, number_of_disk+1)]
    print("initioal_stack:", ini_staks)

    goal_stack= int(input("Which stack end the goal state: "))
    staks[goal_stack][1] = [i for i in range(1, number_of_disk+1)]
    print("goal_stack:", staks)
    
    return ini_staks, staks, goal_stack;

In [12]:
def advanced_tower_of_hanoi():
    ans=True
    while ans:
        print("\n")
        print("1.Redefine the initial state  and goal state")
        print("2.Applay UCS ")
        print("3.Applay A* ")
        print("4.Exit/Quit ")
    
        ans=int(input("What would you like to do?\n "))
        if ans==1:
            print("Redefine the initial state  and goal state\n")
           
            print("Enter initioal state and goal state:")
            ini_state, goal, st_end = input_stacks()
            start = Node(ini_state)
            end = Node(goal)
        elif ans==2:
            print(" Applay UCS\n")
            path_s = ucs(start, end)
        elif ans==3:
            print(" Applay A*\n")
            path_s = A_star_tree(start, end, st_end)
        elif ans==4:
            print("Goodbye\n") 
            ans = False
        else:
            print("\n Not Valid Choice Try again \n")   

In [13]:
advanced_tower_of_hanoi()



1.Redefine the initial state  and goal state
2.Applay UCS 
3.Applay A* 
4.Exit/Quit 
What would you like to do?
 1
Redefine the initial state  and goal state

Enter initioal state and goal state:
Enter the number of stacks: 6
Enter the number of disks: 5
Enter cost: 1
Enter cost: 1
Enter cost: 2
Enter cost: 1
Enter cost: 1
Enter cost: 1
[[0, [], 1], [1, [], 1], [2, [], 2], [3, [], 1], [4, [], 1], [5, [], 1]]
[id of stak, [disk], cost input or output] ==  [0, [], 1]
Which stack begins the initial state: 0
initioal_stack: [[0, [1, 2, 3, 4, 5], 1], [1, [], 1], [2, [], 2], [3, [], 1], [4, [], 1], [5, [], 1]]
Which stack end the goal state: 5
goal_stack: [[0, [], 1], [1, [], 1], [2, [], 2], [3, [], 1], [4, [], 1], [5, [1, 2, 3, 4, 5], 1]]


1.Redefine the initial state  and goal state
2.Applay UCS 
3.Applay A* 
4.Exit/Quit 
What would you like to do?
 3
 Applay A*

***************************
The goal state has been successfully reached
the number expanded node: 38 the cost: 10
the state 

In [14]:
advanced_tower_of_hanoi()



1.Redefine the initial state  and goal state
2.Applay UCS 
3.Applay A* 
4.Exit/Quit 
What would you like to do?
 1
Redefine the initial state  and goal state

Enter initioal state and goal state:
Enter the number of stacks: 5
Enter the number of disks: 4
Enter cost: 1
Enter cost: 2
Enter cost: 1
Enter cost: 1
Enter cost: 1
[[0, [], 1], [1, [], 2], [2, [], 1], [3, [], 1], [4, [], 1]]
[id of stak, [disk], cost input or output] ==  [0, [], 1]
Which stack begins the initial state: 0
initioal_stack: [[0, [1, 2, 3, 4], 1], [1, [], 2], [2, [], 1], [3, [], 1], [4, [], 1]]
Which stack end the goal state: 4
goal_stack: [[0, [], 1], [1, [], 2], [2, [], 1], [3, [], 1], [4, [1, 2, 3, 4], 1]]


1.Redefine the initial state  and goal state
2.Applay UCS 
3.Applay A* 
4.Exit/Quit 
What would you like to do?
 3
 Applay A*

***************************
The goal state has been successfully reached
the number expanded node: 22 the cost: 8
the state start: [[0, [1, 2, 3, 4], 1], [1, [], 2], [2, [], 1], [3,

In [17]:
advanced_tower_of_hanoi()



1.Redefine the initial state  and goal state
2.Applay UCS 
3.Applay A* 
4.Exit/Quit 
What would you like to do?
 1
Redefine the initial state  and goal state

Enter initioal state and goal state:
Enter the number of stacks: 4
Enter the number of disks: 3
Enter cost: 1
Enter cost: 1
Enter cost: 2
Enter cost: 1
[[0, [], 1], [1, [], 1], [2, [], 2], [3, [], 1]]
[id of stak, [disk], cost input or output] ==  [0, [], 1]
Which stack begins the initial state: 0
initioal_stack: [[0, [1, 2, 3], 1], [1, [], 1], [2, [], 2], [3, [], 1]]
Which stack end the goal state: 3
goal_stack: [[0, [], 1], [1, [], 1], [2, [], 2], [3, [1, 2, 3], 1]]


1.Redefine the initial state  and goal state
2.Applay UCS 
3.Applay A* 
4.Exit/Quit 
What would you like to do?
 2
 Applay UCS

***************************
The goal state has been successfully reached
the number expanded node: 61 the cost: 12
the state start: [[0, [1, 2, 3], 1], [1, [], 1], [2, [], 2], [3, [], 1]]
the curent state is a goal: [[0, [], 1], [1, [],

In [20]:
advanced_tower_of_hanoi()



1.Redefine the initial state  and goal state
2.Applay UCS 
3.Applay A* 
4.Exit/Quit 
What would you like to do?
 1
Redefine the initial state  and goal state

Enter initioal state and goal state:
Enter the number of stacks: 4
Enter the number of disks: 5
Enter cost: 1
Enter cost: 2
Enter cost: 1
Enter cost: 1
[[0, [], 1], [1, [], 2], [2, [], 1], [3, [], 1]]
[id of stak, [disk], cost input or output] ==  [0, [], 1]
Which stack begins the initial state: 0
initioal_stack: [[0, [1, 2, 3, 4, 5], 1], [1, [], 2], [2, [], 1], [3, [], 1]]
Which stack end the goal state: 3
goal_stack: [[0, [], 1], [1, [], 2], [2, [], 1], [3, [1, 2, 3, 4, 5], 1]]


1.Redefine the initial state  and goal state
2.Applay UCS 
3.Applay A* 
4.Exit/Quit 
What would you like to do?
 2
 Applay UCS

***************************
The goal state has been successfully reached
the number expanded node: 1019 the cost: 30
the state start: [[0, [1, 2, 3, 4, 5], 1], [1, [], 2], [2, [], 1], [3, [], 1]]
the curent state is a goal: 